In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Define the geometries and yielding indices
geometries = ['NDBR50', 'NDBR20', 'NDBR6', 'CHD6']
dataPoints = {'NDBR50': 50000, 'NDBR20': 50000, 'NDBR6': 50000, 'CHD6': 50000}

# Load data from CSV files
X_files = {
    'NDBR50': "MODEL_DATA/NEWDATA/newData_NDBR50_FD.csv",
    'NDBR20': "MODEL_DATA/NEWDATA/newData_NDBR20_FD.csv",
    'NDBR6': "MODEL_DATA/NEWDATA/newData_NDBR6_FD.csv",
    'CHD6': "MODEL_DATA/NEWDATA/newData_CHD6_FD.csv"
}

Y_file = "MODEL_DATA/NEWDATA/newData_expanded_realHardParam.csv"

X_data = {geometry: pd.read_csv(X_files[geometry], usecols=[0, 1]).values for geometry in geometries}
Y_data = pd.read_csv(Y_file).values

input_size = 2
output_size = 7  # Number of output dimensions (c1 to c7)

# Define a custom Ridge/Lasso regression model
class RidgeLassoRegression(nn.Module):
    def __init__(self, input_size, output_size, alpha=1.0, ridge=True):
        super(RidgeLassoRegression, self).__init__()
        self.fc = nn.Linear(input_size, output_size)  # Single linear layer
        self.alpha = alpha
        self.ridge = ridge

    def forward(self, x):
        x = self.fc(x)
        return x

# Choose between Ridge and Lasso by setting the alpha parameter
ridge_model = RidgeLassoRegression(input_size, output_size, alpha=1.0, ridge=True)  # For Ridge regression
lasso_model = RidgeLassoRegression(input_size, output_size, alpha=1.0, ridge=False)  # For Lasso regression

# Define a loss function (MSE loss) and an optimizer (e.g., Adam)
criterion = nn.MSELoss()
optimizer_ridge = optim.Adam(ridge_model.parameters(), lr=0.001)
optimizer_lasso = optim.Adam(lasso_model.parameters(), lr=0.001)
num_epochs = 1000

# Training loop for Ridge regression
for geometry in geometries:
    num_samples = min(dataPoints[geometry], len(Y_data))
    X_sample = torch.tensor(X_data[geometry][:num_samples], dtype=torch.float32)
    Y_sample = torch.tensor(Y_data[:num_samples], dtype=torch.float32)

    for epoch in range(num_epochs):
        # Forward pass
        outputs = ridge_model(X_sample)

        # Compute the loss with L2 regularization (Ridge)
        loss = criterion(outputs, Y_sample) + ridge_model.alpha * torch.sum(ridge_model.fc.weight ** 2)

        # Backpropagation and optimization
        optimizer_ridge.zero_grad()
        loss.backward()
        optimizer_ridge.step()

        if (epoch + 1) % 100 == 0:
            print(f'Ridge Regression - Geometry: {geometry}, Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Training loop for Lasso regression
for geometry in geometries:
    num_samples = min(dataPoints[geometry], len(Y_data))
    X_sample = torch.tensor(X_data[geometry][:num_samples], dtype=torch.float32)
    Y_sample = torch.tensor(Y_data[:num_samples], dtype=torch.float32)

    for epoch in range(num_epochs):
        # Forward pass
        outputs = lasso_model(X_sample)

        # Compute the loss with L1 regularization (Lasso)
        loss = criterion(outputs, Y_sample) + lasso_model.alpha * torch.sum(torch.abs(lasso_model.fc.weight))

        # Backpropagation and optimization
        optimizer_lasso.zero_grad()
        loss.backward()
        optimizer_lasso.step()

        if (epoch + 1) % 100 == 0:
            print(f'Lasso Regression - Geometry: {geometry}, Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Save the trained Ridge and Lasso regression models
torch.save(ridge_model.state_dict(), 'trained_ridge_model.pth')
torch.save(lasso_model.state_dict(), 'trained_lasso_model.pth')

# Load the trained Ridge and Lasso models for prediction
ridge_model.load_state_dict(torch.load('trained_ridge_model.pth'))
lasso_model.load_state_dict(torch.load('trained_lasso_model.pth'))
ridge_model.eval()
lasso_model.eval()

# Continue with predictions as you did before for linear regression


Ridge Regression - Geometry: NDBR50, Epoch [100/1000], Loss: 17859380.0000
Ridge Regression - Geometry: NDBR50, Epoch [200/1000], Loss: 11714936.0000
Ridge Regression - Geometry: NDBR50, Epoch [300/1000], Loss: 7586885.5000
Ridge Regression - Geometry: NDBR50, Epoch [400/1000], Loss: 4851869.0000
Ridge Regression - Geometry: NDBR50, Epoch [500/1000], Loss: 3059101.5000
Ridge Regression - Geometry: NDBR50, Epoch [600/1000], Loss: 1900450.7500
Ridge Regression - Geometry: NDBR50, Epoch [700/1000], Loss: 1167657.7500
Ridge Regression - Geometry: NDBR50, Epoch [800/1000], Loss: 718254.4375
Ridge Regression - Geometry: NDBR50, Epoch [900/1000], Loss: 453298.3750
Ridge Regression - Geometry: NDBR50, Epoch [1000/1000], Loss: 304121.5938
Ridge Regression - Geometry: NDBR20, Epoch [100/1000], Loss: 223749.0000
Ridge Regression - Geometry: NDBR20, Epoch [200/1000], Loss: 182094.4688
Ridge Regression - Geometry: NDBR20, Epoch [300/1000], Loss: 162661.5625
Ridge Regression - Geometry: NDBR20, Epoc

RidgeLassoRegression(
  (fc): Linear(in_features=2, out_features=7, bias=True)
)